In [1]:
import psycopg2
import sys
import pandas as pd

In [2]:
cs = "host='localhost' port='5432' dbname='datamodel' user='postgres' password='admin'"
conn = psycopg2.connect(cs)

In [3]:
conn = psycopg2.connect(cs)
cursor = conn.cursor()

# Set Archimate model name
archimate_model = 'Data model'

cursor.execute("""
SELECT E.name,  P.name, P.value FROM elements E
INNER JOIN properties P ON E.id = P.parent_id
WHERE E.version=P.parent_version
AND P.parent_version = (SELECT MAX(parent_version) FROM properties 
INNER JOIN elements ON elements.id = P.parent_id 
WHERE elements.version=properties.parent_version)
""")
rows = cursor.fetchall()

In [4]:
#import matplotlib.pyplot as plt
#%matplotlib inline

from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.charts import Bar, Donut, defaults, Scatter, TimeSeries, Line, Area, color, marker
from bokeh.layouts import row, column
output_notebook()

data = pd.DataFrame(rows, columns=['element_name', 'property', 'quality'])
data['property_name'], data['date'] = data['property'].str.split(' ',1).str #, expand=True)

data['date']=pd.to_datetime(pd.Series(data['date']))
data['quality'] = data['quality'].astype('float')
data.index = data['date']

data['ep'] = data['element_name'] + '|' + data['property_name']
eps = data['ep'].unique()

graphs_data = [data[(data['element_name']==d.split('|',1)[0]) & (data['property_name']==d.split('|',1)[1])]
[['date', 'element_name','property_name', 'quality']] for d in eps]

graphs = [TimeSeries(gd.sort_index(), x='date', y='quality', 
                title=gd['element_name'][0], ylabel=gd['property_name'][0], xlabel="Date") for gd in graphs_data]
plot = column(graphs)

show(plot)

Loading BokehJS ...

c:\python27\lib\site-packages\bokeh\core\json_encoder.py:73: FutureWarning: pandas.tslib is deprecated and will be removed in a future version.
You can access Timestamp as pandas.Timestamp
  if pd and isinstance(obj, pd.tslib.Timestamp):
